# AIMO 3 Competition Submission

This notebook uses the ToolOrchestra-based multi-domain solver to solve AIMO 3 problems.

## System Overview
- **7 Domain Solvers**: Geometry (25 theorems), Algebra, Number Theory, Combinatorics, Graph Theory, Analysis, Symbolic
- **ToolOrchestra Integration**: RL-based intelligent tool orchestration
- **Stability Tracking**: Optional (disabled for speed in competition)

## Competition Requirements
- CPU Notebook: ≤ 9 hours runtime
- GPU Notebook: ≤ 5 hours runtime
- Internet: Disabled
- Submission file: Generated by API


## Setup and Imports


In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path
# In Kaggle, working directory is /kaggle/working
project_root = Path('/kaggle/working')
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add kaggle_evaluation to path if needed
kaggle_eval_path = Path('/kaggle/input/ai-mathematical-olympiad-progress-prize-3/kaggle_evaluation')
if kaggle_eval_path.exists() and str(kaggle_eval_path) not in sys.path:
    sys.path.insert(0, str(kaggle_eval_path.parent))

# Import Kaggle evaluation API
try:
    import kaggle_evaluation.core.templates
    import aimo_3_gateway
except ImportError:
    # Fallback: try importing from local path
    import importlib.util
    gateway_path = project_root / 'ai-mathematical-olympiad-progress-prize-3' / 'kaggle_evaluation' / 'aimo_3_gateway.py'
    if gateway_path.exists():
        spec = importlib.util.spec_from_file_location('aimo_3_gateway', gateway_path)
        aimo_3_gateway = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(aimo_3_gateway)
    
    templates_path = project_root / 'ai-mathematical-olympiad-progress-prize-3' / 'kaggle_evaluation' / 'core' / 'templates.py'
    if templates_path.exists():
        spec = importlib.util.spec_from_file_location('templates', templates_path)
        templates = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(templates)
        kaggle_evaluation = type('kaggle_evaluation', (), {'core': type('core', (), {'templates': templates})()})()

# Import our solver
from src.orchestration import create_aimo_orchestrator

print("✓ Imports successful")


## Initialize Solver


In [ ]:
# Initialize ToolOrchestra adapter (best solver)
# Disable stability tracking for speed in competition
orchestrator = create_aimo_orchestrator(
    use_toolorchestra=True,
    measure_stability=False,  # Disabled for speed
    track_orchestration_stability=False,  # Disabled for speed
)

print("✓ Solver initialized")


## Define Predict Function

The Kaggle evaluation API calls a `predict` function with problem data.
We need to extract the problem statement and return the answer.


In [ ]:
import polars as pl
from typing import Any

def predict(data_batch: pl.DataFrame) -> pl.DataFrame:
    """
    Predict function called by Kaggle evaluation API.
    
    Args:
        data_batch: DataFrame with 'id' and 'problem' columns
        
    Returns:
        DataFrame with 'id' and 'answer' columns
    """
    results = []
    
    for row in data_batch.iter_rows(named=True):
        problem_id = row['id']
        problem_statement = row['problem']
        
        try:
            # Solve the problem
            answer = orchestrator.solve(problem_statement, problem_id=problem_id)
            
            # Ensure answer is in valid range [0, 99999]
            answer = max(0, min(99999, int(answer)))
            
        except Exception as e:
            # Fallback to 0 on error
            print(f"Error solving {problem_id}: {e}")
            answer = 0
        
        results.append({
            'id': problem_id,
            'answer': answer
        })
    
    # Return as DataFrame
    return pl.DataFrame(results)


## Initialize Inference Server


In [ ]:
class AIMO3InferenceServer(kaggle_evaluation.core.templates.InferenceServer):
    """Inference server for AIMO 3 competition."""
    
    def _get_gateway_for_test(self, data_paths=None, *args, **kwargs):
        """Get gateway for local testing."""
        return aimo_3_gateway.AIMO3Gateway(data_paths)

# Create inference server with our predict function
inference_server = AIMO3InferenceServer(predict)

print("✓ Inference server initialized")


## Run Evaluation

In Kaggle environment, this will automatically:
1. Receive problems from the Gateway
2. Call our predict function
3. Generate submission file

**Note:** This cell will block until all problems are processed.


In [ ]:
# Start the inference server
# In Kaggle environment, this will run until all problems are solved
inference_server.serve()


## Local Testing (Optional)

For local testing, you can run the gateway manually:


In [ ]:
# Uncomment to test locally
# inference_server.run_local_gateway(
#     data_paths=('/kaggle/input/ai-mathematical-olympiad-progress-prize-3/test.csv',)
# )
